In [38]:
import math
import scipy as sp
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt

from sklearn import gaussian_process
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel, RBF
from random import Random
from numpy import *
from time import *
from datetime import *
from IPython.display import display
from time import time

%matplotlib inline
%load_ext autoreload
%autoreload 2


for testing_matrix in range(6, 6+7):

    x_train = np.array(np.linspace(0.01,1.0, testing_matrix**3), dtype='float32').reshape(testing_matrix**3, 1)
    
    y_train = []

    for idx in range(len(x_train)):
        y_train.append(random.uniform(0, 10))
        #y_train.append(1)
    y_train = np.array(y_train)

    sk_kernel = RBF(length_scale=1.0, length_scale_bounds=(0.01, 10000.0)) #+ WhiteKernel(noise_level = 1e-12) #100000.0
    gp = gaussian_process.GaussianProcessRegressor(kernel=sk_kernel, n_restarts_optimizer=20)
    gp.fit(x_train,y_train)
    y_pred, sigma_hat = gp.predict(x_test, return_std=True)

    l = gp.kernel_.length_scale
    rbf= gaussian_process.kernels.RBF(length_scale=l)

    Kernel= rbf(x_train, x_train)
    K_y = Kernel + np.eye(testing_matrix**3) * 1e-8
        
    t1 = time()
    #L, lower = sp.linalg.cho_factor(K_y, lower=True)
    L1 = sp.linalg.cholesky(K_y, lower=True)
    t2 = time()
    runtimeFT = t2 - t1
    
    print(" ")
    print("Matrix size is {size}.".format(size=L1.shape))
    print("Relative error of the factorization is {error}".format(error=np.linalg.norm(L1@L1.T-K_y)/np.linalg.norm(K_y)))
    print("Runtime of factorization is {rt} seconds.".format(rt=runtimeFT))
    print(" ")
    
    L1 = L1, lower
    b = np.ones(testing_matrix**3)
    c = K_y@b.T
    t3 = time()
    sol = sp.linalg.cho_solve(L1, c.T) 
    ##### cho_solve use result of cho_factor, therefore to be relatable between cho_solve and cholesky, 
    ##### the equivalence between cholesky and cho_factor needs to be verified.
    t4 = time()
    runtimeLS = t4 - t3 
    
    print("Relative error of solving the LS is {error}".format(error=np.linalg.norm(sol-b)/np.linalg.norm(b)))
    print("Runtime of solving the LS is {rt} seconds.".format(rt=runtimeLS))
    print(" ")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


C:\Users\Woodrow\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


 
Matrix size is (216, 216).
Relative error of the factorization is 1.381768403444049e-16
Runtime of factorization is 0.0010161399841308594 seconds.
 
Relative error of solving the LS is 2.22950855480604e-08
Runtime of solving the LS is 0.0 seconds.
 
 
Matrix size is (343, 343).
Relative error of the factorization is 1.7007239692648267e-16
Runtime of factorization is 0.002994060516357422 seconds.
 
Relative error of solving the LS is 1.3972304578105185e-07
Runtime of solving the LS is 0.0 seconds.
 
 
Matrix size is (512, 512).
Relative error of the factorization is 1.9038599692753856e-16
Runtime of factorization is 0.010993480682373047 seconds.
 
Relative error of solving the LS is 2.233839947742477e-07
Runtime of solving the LS is 0.0010025501251220703 seconds.
 
 
Matrix size is (729, 729).
Relative error of the factorization is 1.7159446608894352e-16
Runtime of factorization is 0.02000713348388672 seconds.
 
Relative error of solving the LS is 5.812107812370741e-07
Runtime of solv